<a href="https://colab.research.google.com/github/adithya-badidey/ActiveLearning/blob/main/06_baal_brazilian_ndti.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [2]:
! pip install baal
! pip install segmentation_models_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 61 kB 102 kB/s 
     |████████████████████████████████| 419 kB 9.4 MB/s 
     |████████████████████████████████| 53 kB 2.3 MB/s 
     |████████████████████████████████| 9.4 MB 95.7 MB/s 
     |████████████████████████████████| 4.5 MB 78.6 MB/s 
     |████████████████████████████████| 295 kB 88.7 MB/s 
     |████████████████████████████████| 965 kB 83.6 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 4.5 MB/s 
     |████████████████████████████████| 376 kB 23.9 MB/s 
     |████████████████████████████████| 58 kB 8.3 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=06d87863c406e6592cce02c2d76763cb0ce939777c144d693ddaa8dd2893c0db
  Stored in directory: /root/.cache/pip/wheels/84/b9/90/25a0195cf95fb5533db96f1c77ea3f296b7cc86ae8ae48e3dc
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60966 sha256=91badedf0100d615e86b9c2397bd1cd62ffdebc18edbb0635815de306bbb9e34
  Stored in directory: /root/.cache/pip/wheels/ed/fa/b9/5c82b59d905f95542a192b883c0cc0082407ea2f54beb2f9e6
Successfully built efficientnet-pytorch pretrainedmodels


In [3]:
import torchvision

In [4]:
import argparse

import baal
from baal import ActiveLearningDataset
from baal.active import get_heuristic, ActiveLearningLoop
from baal.bayesian.dropout import MCDropoutModule
from copy import deepcopy

import numpy as np

import segmentation_models_pytorch as smp

from PIL import Image
from pprint import pprint

import torch
from torch import nn
from torch.nn import functional as F

from torchvision.transforms import transforms
from torch.cuda.amp import GradScaler, autocast


from tqdm import tqdm

from typing import List


In [5]:
debug=False

print("Pytorch: \t\t", torch.__version__)
if not debug and torch.cuda.is_available():
    device='cuda'
    print('GPU:         \t\t', torch.cuda.get_device_name(0))
    print('Memory Usage:\t',
        round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB / ',
        round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
    
    torch.backends.cudnn.benchmark = True
else:
    print("GPU is **not available**")
    device='cpu'

Pytorch: 		 1.13.0+cu116
GPU:         		 A100-SXM4-40GB
Memory Usage:	 0.0 GB /  0.0 GB


In [6]:
# Initialization
%load_ext autoreload
%autoreload 2
import sys
import os
import pathlib

# To be able to reference packages/modules in this repository, this
# relative path must be added to the python path. Your notebook may be 
# in a different folder, so modify this variable to point to the src 
# folder.
proj_notebooks_root = pathlib.Path().absolute()
proj_root_path = proj_notebooks_root
data_path = proj_notebooks_root / "data"

if proj_root_path not in sys.path:
    sys.path.insert(0, proj_root_path.as_posix())
    print("Updated Python Path")

print(f"Project Root Path: {proj_root_path}")
print(f"Project Source Root Path: {proj_notebooks_root}")
print(f"Project Data Path: {data_path}")

Updated Python Path
Project Root Path: /content
Project Source Root Path: /content
Project Data Path: /content/data


In [7]:
base_dir = data_path / 'biankatpas' / 'Dataset'

if not base_dir.exists():
    dataset_url = 'https://github.com/biankatpas/Cracks-and-Potholes-in-Road-Images-Dataset/tarball/master'

    ! wget {dataset_url} -P {data_path}
    ! tar xvf {data_path / 'master'} -C {data_path}
    ! mv {data_path / 'biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054'} {data_path / 'biankatpas'}
    ! rm {data_path / 'master'}
else:
    print("Found the dataset")

Streaming output truncated to the last 5000 lines.
biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset/1092998_DF_251_251BDF0052_00941/1092998_DF_251_251BDF0052_00941_RAW.jpg
biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset/1092999_DF_251_251BDF0052_00943/
biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset/1092999_DF_251_251BDF0052_00943/1092999_DF_251_251BDF0052_00943_CRACK.png
biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset/1092999_DF_251_251BDF0052_00943/1092999_DF_251_251BDF0052_00943_LANE.png
biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset/1092999_DF_251_251BDF0052_00943/1092999_DF_251_251BDF0052_00943_POTHOLE.png
biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset/1092999_DF_251_251BDF0052_00943/1092999_DF_251_251BDF0052_00943_RAW.jpg
biankatpas-Cracks-and-Potholes-in-Road-Images-Dataset-1f20054/Dataset/1093000_DF_251_251BDF0052_00945/
biankatpas-Cracks-and-Poth

In [8]:
image_suffix = '_RAW.jpg'
segment_suffix = {
    'lane':'_LANE.png',
    'pothole':'_POTHOLE.png',
    'crack':'_CRACK.png',
}

def getSegment(directory, segm):
    segment = Image.open(directory / (directory.stem+segment_suffix[segm]))
    segment = np.array(segment)
    segment = segment[:,:,0]/255
    return segment

# Counting the number of directories and checking the number of files
# inside each. Also, the images are in slightly different dimensions.
# We shall account for this later.
f = [y for y in (base_dir).glob('*')] 
counts = {}
image_sizes = {}

for i in f:
    files = [y for y in i.glob('*')] 
    num = len(files)
    counts[num] = counts.get(num, 0) + 1
    image = Image.open(i/(i.stem+image_suffix))
    image_sizes[image.size] = image_sizes.get(image.size, 0) + 1
    
print(f"In {base_dir}:")
for count in counts:
    print(f"Found {counts[count]} dirs with {count} files each.")

print()

print(f"{'Resolution':<12}| {'Count':<10}")
print('-' * 23)
for i in image_sizes:
    print(f"{str(i):<12}| {image_sizes[i]:<10}")

In /content/data/biankatpas/Dataset:
Found 2235 dirs with 4 files each.

Resolution  | Count     
-----------------------
(1024, 640) | 1227      
(1024, 630) | 1008      


In [9]:
def getAllSegments(directory):
    segment_lane = getSegment(directory, 'lane')
    segment_crack = getSegment(directory, 'crack')
    segment_potholes = getSegment(directory, 'pothole')

    segment_crack = segment_crack - (segment_potholes)
    segment_crack = np.clip(segment_crack, 0,1)

    segment_lane = segment_lane - (segment_crack + segment_potholes)
    segment_lane = np.clip(segment_lane, 0,1)

    segment_bg = np.ones_like(segment_potholes) - (segment_lane + segment_crack + segment_potholes)
    segment_bg = np.clip(segment_bg, 0,1)

    segment_arrs = np.stack([segment_bg, segment_lane, segment_potholes, segment_crack], -1)
    return segment_arrs


# class CracksDataset(torch.utils.data.Dataset):
#     def __init__(self, files,image_transforms=None, both_transforms=None):
#         self.files = files
        
#         self.image_transforms = image_transforms
#         self.both_transforms = both_transforms
    
#     def __len__(self):
#         return len(self.files)

#     def __getitem__(self, idx):
#         chosenDir = self.files[idx]
#         image = Image.open(chosenDir/(chosenDir.stem+image_suffix))
#         segments = torch.Tensor(getAllSegments(chosenDir).transpose(2,0,1))
            
#         if self.image_transforms is not None:
#             image = self.image_transforms(image)
            
#         if self.both_transforms is not None:
#             image, segments = self.both_transforms(image, segments)
        
#         return image, segments
        
# numclasses = 4
# class_names=['lanes', 'potholes', 'cracks']

In [10]:
import torchvision.transforms.functional as F

class CenterCropBoth(torch.nn.Module):
    def __init__(self, size):
        super().__init__()
        self.size = size

    def forward(self, img, mask):
        return F.center_crop(img, self.size), F.center_crop(mask, self.size)

class RandomHorizontalFlipBoth(torch.nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p = p

    def forward(self, img, mask):
        if torch.rand(1) < self.p:
            return F.hflip(img), F.hflip(mask)
        return img, mask
    
class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, *img):
        for t in self.transforms:
            img = t(*img)
        return img

data_transforms = {
    'both': Compose([                # This uses our custom Compose class
        RandomHorizontalFlipBoth()
    ]),
    'images': transforms.Compose([   # This uses the default torchvision.transforms.Compose
        transforms.CenterCrop((630, 1024)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

def mean_regions(n, grid_size=16):
    # Compute the mean uncertainty per regions.
    # [batch_size, W, H]
    n = torch.from_numpy(n[:, None, ...])
    # [Batch_size, 1, grid, grid]
    out = F.adaptive_avg_pool2d(n, grid_size)
    return np.mean(out.view([-1, grid_size**2]).numpy(), -1)


In [11]:
class ArrayDataset(torch.utils.data.Dataset):
    def __init__(self, array, image_transforms=None, both_transforms=None):
        self.array = array
                
        self.image_transforms = image_transforms
        self.both_transforms = both_transforms
        self.segments_transforms = transforms.Compose([ 
          transforms.CenterCrop((630, 1024))
        ])

    def __len__(self):
        return len(self.array)

    def __getitem__(self, index):
          chosenDir = self.array[index]
          image = Image.open(chosenDir/(chosenDir.stem+image_suffix))
          segments = torch.Tensor(getAllSegments(chosenDir).transpose(2,0,1))
          segments = self.segments_transforms(segments)
              
          if self.image_transforms is not None:
              image = self.image_transforms(image)
              
          if self.both_transforms is not None:
              image, segments = self.both_transforms(image, segments)
          
          return image, segments
    
    def split(self, p=0.5):
        count = len(self.array)
        index = np.arange(count)
        first = int(count * p)
        return [
            ArrayDataset(self.array[index[:first]], 
                    image_transforms=self.image_transforms, 
                    both_transforms=self.both_transforms),
            ArrayDataset(self.array[index[first:]], 
                    image_transforms=self.image_transforms, 
                    both_transforms=self.both_transforms)
        ]
    
    def split_count(self, first):
        return [
            ArrayDataset(self.array[:first], 
                    image_transforms=self.image_transforms, 
                    both_transforms=self.both_transforms), 
            ArrayDataset(self.array[first:], 
                    image_transforms=self.image_transforms, 
                    both_transforms=self.both_transforms)
        ]

def get_datasets(initial_pool):
    all_data = [y for y in (base_dir).glob('*')] 

    dataset = ArrayDataset(all_data,
                            image_transforms=data_transforms['images'], 
                            both_transforms=data_transforms['both'])


    test_set, active_set = dataset.split_count(initial_pool)
    print("Active Set: ", len(active_set))
    print("Test Set: ", len(test_set))
    
    active_set = ActiveLearningDataset(active_set,
                           pool_specifics={'both_transforms': None})
    
    active_set.label_randomly(initial_pool)
    return active_set, test_set

In [12]:
active_set, _ = get_datasets(30)
active_set[0]
# print(active_set.image_transforms)

Active Set:  2205
Test Set:  30


(tensor([[[ 1.8550,  1.8893,  1.8722,  ..., -1.2103, -1.2274, -1.1418],
          [ 1.7865,  1.7865,  1.7523,  ..., -1.2617, -1.2274, -1.0904],
          [ 1.7865,  1.7523,  1.6667,  ..., -1.0904, -0.9534, -0.7137],
          ...,
          [ 0.1768,  0.1426,  0.0912,  ..., -0.4568, -0.4054, -0.3883],
          [ 0.0569,  0.0398,  0.0227,  ..., -0.3369, -0.3027, -0.3027],
          [-0.0458, -0.0287, -0.0287,  ..., -0.3369, -0.3369, -0.3369]],
 
         [[ 2.1835,  2.2535,  2.2885,  ..., -0.9503, -0.9503, -0.8102],
          [ 2.1835,  2.2010,  2.2010,  ..., -1.0028, -0.9503, -0.8102],
          [ 2.2535,  2.2185,  2.1485,  ..., -0.8978, -0.7052, -0.4601],
          ...,
          [ 0.1877,  0.1527,  0.1001,  ..., -0.3200, -0.2675, -0.2500],
          [ 0.1001,  0.0826,  0.0651,  ..., -0.1975, -0.1625, -0.1625],
          [-0.0049,  0.0126,  0.0126,  ..., -0.1975, -0.1975, -0.1975]],
 
         [[ 2.4831,  2.5703,  2.6400,  ..., -1.1944, -1.1944, -1.0724],
          [ 2.5006,  2.5703,

In [13]:
import matplotlib.pyplot as plt

active_set, _ = get_datasets(30)
dataloader = torch.utils.data.DataLoader(active_set,
                                        batch_size=5,
                                        shuffle=False,
                                        num_workers=1)
batch = next(iter(dataloader))
tensors = batch[0]
segments = batch[1]
print("Batch's image tensor size: ", tensors.shape)

# tensors, metadata
y = 5
x = 2
fig, axs = plt.subplots(y, x, 
                        sharey=True,
                        sharex=True,
                        tight_layout=True, 
                        figsize=(4*x, 2.5*y))

for i in range(y):
    ax = axs[i][0]
    if i == 0:
        ax.set_title('Input Image')
    image = tensors[i].permute(1,2,0).cpu().numpy()
    image = (image - [image[:,:,0].min(), image[:,:,1].min(), image[:,:,2].min()])
    image = image/([image[:,:,0].max(), image[:,:,1].max(), image[:,:,2].max()])
    ax.set_axis_off()
    ax.imshow(image)
    
    ax = axs[i][1]
    if i == 0:
        ax.set_title('Gold')
    res = segments[i][0] + \
            2 * segments[i][1] + \
            4 * segments[i][2] + \
            8 * segments[i][3]
    ax.set_axis_off()
    ax.imshow(res, cmap='viridis', interpolation='nearest', vmin=0, vmax=8)

dataloader = None

Active Set:  2205
Test Set:  30
Batch's image tensor size:  torch.Size([5, 3, 630, 1024])


ImportError: ignored

<Figure size 576x900 with 10 Axes>

In [14]:
p_active_learning_steps = 3

p_initial_pool = 250
p_query_size = 250
p_query_interations = 20

p_heuristic = "random"
p_reduce="sum"

p_learning_epochs=10
p_batch_size = 12
p_learning_rate = 0.001

num_classes = 4
class_names=['lanes', 'potholes', 'cracks']

use_cuda = torch.cuda.is_available()
# use_cuda = False
print("Use Cuda:", use_cuda)

Use Cuda: True


In [15]:
def get_eval_metrics(pred, gold, threshold = 0.5):
    pred = (pred > threshold).long()
    hits = torch.sum(torch.mul(pred,gold)).item() #element-wise multiplication
    shots = torch.sum(pred).item()
    targets = torch.sum(gold).item()
    return hits, shots, targets


def train_loop(dataloader, model, loss_fn, optimizer, history, lr_sched = None):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)

    model.train()
    
    total_loss = 0
    for X, y in dataloader:
        X = X.to(device)
        y = y.to(device)
        optimizer.zero_grad()

        # Runs the forward pass with autocasting.
        with autocast():
            output = model(X)
            loss = loss_fn(output, y)

        # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
        # Backward passes under autocast are not recommended.
        # Backward ops run in the same dtype autocast chose for corresponding forward ops.
        scaler.scale(loss).backward()

        # scaler.step() first unscales the gradients of the optimizer's assigned params.
        # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
        # otherwise, optimizer.step() is skipped.
        scaler.step(optimizer)

        # Updates the scale for next iteration.
        scaler.update()

        if(lr_sched is not None):
            lr_sched.step()

        total_loss += loss.item()
    history['train_loss'].append(total_loss/num_batches)

    
def test_loop(dataloader, model, loss_fn, history):
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    loss = 0

    accuracy_metrics = {}
    for i in class_names:
        accuracy_metrics[i] = {'hits':0, 'shots':0, 'targets':0}
    
    with torch.no_grad():
        for X, y in dataloader:
            X = X.to(device)
            y = y.to(device)
            
            output = model(X)
                        
            for i in range(len(class_names)):
                c = class_names[i]
                hits, shots, targets = get_eval_metrics(output[:,i+1,:,:], y[:,i+1,:,:])
                accuracy_metrics[c]['hits'] += hits
                accuracy_metrics[c]['shots'] += shots
                accuracy_metrics[c]['targets'] += targets
    
    for category in accuracy_metrics:
        i = accuracy_metrics[category]
        shots = i['shots']
        hits = i['hits']
        targets = i['targets']
        precision = 0.
        if shots > 0:
            precision = i['hits']/i['shots']
        i['precision'] = precision
        
        recall = 0.
        if targets > 0:
            recall = i['hits']/i['targets']
        i['recall'] = recall
        
        f1score = 0.
        if precision > 0 and recall > 0:
            f1score = (2 * precision * recall)/(precision + recall)
        i['f1score'] = f1score
        
    loss /= num_batches
    history['test_loss'].append(loss)
    history['accuracy_metrics'].append(accuracy_metrics)
    


def train_model(model_wrapper, loss_fn, train_dataset, test_dataset,epochs=40):

    history = {
        'train_lr':[],
        'train_loss':[],
        'test_loss':[],
        'accuracy_metrics':[]
    }

    model_wrapper.model
    train_dataloader = torch.utils.data.DataLoader(train_dataset,
                            batch_size=p_batch_size,
                            shuffle=True,
                            prefetch_factor=2,
                            pin_memory=True,
                            num_workers=12)

    test_dataloader = torch.utils.data.DataLoader(test_dataset,
                            batch_size=p_batch_size,
                            shuffle=False,
                            num_workers=12)
        
    optimizer = torch.optim.SGD(params, lr=0.00001, momentum=0.9)

    lr_sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, 
                          max_lr=0.01, 
                          steps_per_epoch=len(train_dataloader), 
                          epochs=epochs)

    for t in tqdm(range(epochs), bar_format="{elapsed} Elapsed | {percentage:3.0f}% done |{bar}| {n_fmt}/{total_fmt} [{remaining} remaining | {rate_fmt}{postfix}]", unit="epoch", total=epochs):
#         dataloader, model, loss_fn, optimizer, scaler, history=None
        train_loop(train_dataloader, model_wrapper.model, loss_fn, optimizer, history, None)
        test_loop(test_dataloader, model_wrapper.model, loss_fn, history)

        # print(f"Training Loss: {history['train_loss'][-1]:.5}\t    Testing Loss: {history['test_loss'][-1]:.5}")
        # accuracy_metric = history['accuracy_metrics'][-1]
        # for i in accuracy_metric:
        #     print(f"{i:<8}  |   Precision: {accuracy_metric[i]['precision']:8.5f}     Recall: {accuracy_metric[i]['recall']:8.5f}")
    test_dataloader = None
    train_dataloader = None
    return model, history

In [16]:
model = None

In [17]:

# We will use the FocalLoss
if num_classes > 1:
    class_weights = torch.tensor([1,1,2,4]).to(device)
    criterion = torch.nn.CrossEntropyLoss(class_weights)
else:
    criterion = nn.BCEWithLogitsLoss()

# 
# model = smp.Unet(
#     encoder_name="resnext50_32x4d",
#     encoder_depth=5,
#     encoder_weights="imagenet",
#     decoder_use_batchnorm=False,
#     classes=1,
# )
# add_dropout(model, classes=1, activation=None)

class SegModel(torch.nn.Module):

    def __init__(self, numclasses):
        super(SegModel, self).__init__()

        self.fcn = torchvision.models.segmentation.fcn_resnet50(
            weights=None, 
            num_classes = numclasses, 
            aux_loss = False,
            weights_backbone=torchvision.models.ResNet50_Weights.DEFAULT)

    def forward(self, x):
        x = self.fcn(x)['out']
        return x

model = SegModel(num_classes)
scaler = GradScaler()



params = [p for p in model.parameters() if p.requires_grad]

# This will enable Dropout at test time.
model = MCDropoutModule(model)


# Keep a copy of the original weights
initial_weights = deepcopy(model.state_dict())

# Put everything on GPU.
if use_cuda:
    model.cuda()

# Add metrics
model = baal.ModelWrapper(model, criterion, replicate_in_memory=False)

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
heuristics = ['random', 'entropy', 'bald']
all_histories = []
for h in heuristics:
    print("="*20)
    print(f"Heuristic: {h}")
    print("="*20)
    if h == 'batch_bald':
        heuristic = get_heuristic(h, reduction=mean_regions, num_samples=1000)
    else:
        heuristic = get_heuristic(h, reduction=mean_regions)

    active_set, test_set = get_datasets(p_initial_pool)

    image, segment = active_set[0]
    image.shape, segment.shape

    # The ALLoop is in charge of predicting the uncertainty and
    loop = ActiveLearningLoop(
        active_set,
        model.predict_on_dataset,
        heuristic=heuristic,
        query_size=p_query_size,
        # Instead of predicting on the entire pool, only a subset is used
        max_sample=500,
        batch_size=p_batch_size,
        iterations=p_query_interations,
        use_cuda=use_cuda,
    )
    history = []
    all_histories.append(history)
    for epoch in range(p_active_learning_steps):
        print(f"Step: {epoch + 1}")
        # Following Gal et al. 2016, we reset the weights.
        model.load_state_dict(initial_weights)
        # Train 50 epochs before sampling.

        print("Training Model...")
        model, stats = train_model(model, criterion, 
                                   active_set, test_set, 
                                   epochs=p_learning_epochs)
        history.append(stats)
        print()
        accuracy_metric = stats['accuracy_metrics'][-1]
        for i in accuracy_metric:
            print(f"{i:<8}  |   Precision: {accuracy_metric[i]['precision']:8.5f}     Recall: {accuracy_metric[i]['recall']:8.5f}")
        
        print()

        print("Querying...")

        should_continue = loop.step()

        print("-"*20)

        if not should_continue:
            break
    
    print("Training Final Model...")
    model, stats = train_model(model, criterion, 
                                active_set, test_set, 
                                epochs=p_learning_epochs)
    history.append(stats)
    print()
    print("Last Training:")
    accuracy_metric = stats['accuracy_metrics'][-1]
    for i in accuracy_metric:
        print(f"{i:<8}  |   Precision: {accuracy_metric[i]['precision']:8.5f}     Recall: {accuracy_metric[i]['recall']:8.5f}")
    print("Querying...")
    print()

Heuristic: random
Active Set:  1985
Test Set:  250
Step: 1
Training Model...


03:33 Elapsed | 100% done |██████████| 10/10 [00:00 remaining | 21.36s/epoch]


lanes     |   Precision:  0.68379     Recall:  0.99403
potholes  |   Precision:  0.19500     Recall:  0.10014
cracks    |   Precision:  0.08704     Recall:  0.63275

Querying...
[91-MainThread   ] [baal.modelwrapper:predict_on_dataset_generator:232] 2022-12-17T04:51:54.048171Z [info     ] Start Predict                  dataset=500


 71%|███████▏  | 30/42 [02:26<00:58,  4.88s/it]

In [ ]:
len(all_histories[-1]), all_histories[-1][0]

In [ ]:
len(all_histories), len(all_histories[0]), all_histories[0][0].keys()

In [ ]:
import json
with open('log.txt', 'w') as logfile:
    logfile.write(json.dumps(all_histories))

In [ ]:
import json
 
with open('data.json') as json_file:
    data = json.load(json_file)
 
    # Print the type of data variable
    print("Type:", type(data))
 
    # Print the data of dictionary
    print("\nPeople1:", data['people1'])
    print("\nPeople2:", data['people2'])

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

def plotHistory(history):
    metrics = ['train_loss', 'train_precision', 'train_recall', 'train_f1score',
           'test_loss','test_precision','test_recall', 'test_f1score']
    gradients = ['Greys', 'Purples', 'Blues', 'Greens', 'Oranges', 'Reds',
                          'YlOrBr', 'YlOrRd', 'OrRd', 'PuRd', 'RdPu', 'BuPu',
                          'GnBu', 'PuBu', 'YlGnBu', 'PuBuGn', 'BuGn', 'YlGn']
    norm = matplotlib.colors.Normalize(vmin=0, vmax=len(history))
    rows = 2
    cols = round(len(metrics)/rows)

    fig, axes = plt.subplots(rows, cols, figsize=(12,8), sharey = True)

    for index, metric in enumerate(metrics):
        cmap = matplotlib.cm.get_cmap(gradients[index])

        ax = axes[index//cols,index%cols]
        for i, val in enumerate(history):
            ax.plot(val[metric], color=cmap(norm(i)))
        ax.set_title(metric)

print("For Heuristic: ", heuristics[0])
plotHistory(all_histories[0])
plt.savefig(f'{heuristics[0]}.png', bbox_inches='tight')




In [ ]:
print("For Heuristic: ", heuristics[1])
plotHistory(all_histories[1])
plt.savefig(f'{heuristics[1]}.png', bbox_inches='tight')


In [ ]:
print("For Heuristic: ", heuristics[2])
plotHistory(all_histories[2])
plt.savefig(f'{heuristics[2]}.png', bbox_inches='tight')


In [ ]:
print("For Heuristic: ", heuristics[3])
plotHistory(all_histories[3])
plt.savefig(f'{heuristics[3]}.png', bbox_inches='tight')